<a name="top"></a><img src="source/SpinalHDL.png" alt="SpinalHDL based on Scala" style="width:320px;" />

运行SpinalHDL代码之前，务必加载SpinalHDL的库 
（**注**：*第一次运行时需要下载库，可能会有点慢，请耐心等待*）

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-spinal.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

## Chapter


In [ ]:
case class IQ(width: Int) extends Bundle{
  val I,Q = SInt(width bits)
}
class Myflow1 extends Component{
    val din = slave  Flow(IQ(8))
    val dout   = master Flow(IQ(8))    
    dout <-< din     
    val write = Stream(Fragment(Bits(8 bits)))
}

showRtl(new Myflow1)

In [ ]:
class T1 extends Component{
    val source = slave Stream(Fragment(Bits(8 bits)))
    val sink = master Stream(Fragment(Bits(8 bits)))
 sink << source
}
showRtl(new T1)

In [ ]:
class T2 extends Component{
    val a = in UFix(8 exp, 10 bits)
    val b = in UFix(2 exp, 10 bits)
    val c = out Bits()
c := (a + b).asBits
}
showRtl(new T2)

In [ ]:
class T3 extends Component{
    val a = in UInt(8 bits)
    val b = Bool
    b := a === M"00--11--"
    print(Q"8.2")
}
showRtl(new T3)

In [ ]:
class T3 extends Component{ 
    val paraa = 2*2
    val a = in UInt(6 bits)
    val b = out SInt(paraa bits)
    val remain = a.asSInt-33
    b := remain.resized
}
showRtl(new T3)

In [ ]:
class T3 extends Component{  
    val a = in Bool()
    val en,set,clc = in Bool
    val b = out Bool()
    val cnt = RegNext(a  ) setWhen(set) clearWhen(clc) init False
    b := en && !(set && !clc) 
}
showRtl(new T3)

In [ ]:
case class IQ(width: Int) extends Bundle{
  val I,Q = SInt(width bits)
}
class D extends Component{
    val iq = in(IQ(8)) // Faild too!!!
    val en = in(Bool())
}
showRtl(new D)

In [ ]:
class D extends Component{
    val io = new Bundle{
      val flush = in Bool()
      val iq = in(UInt(8 bits))
      val en = in(Bool())    
        val rd = in Bool()
        val raddr = in UInt(4 bits)        
        val rdata = out UInt(8 bits)
    }    
    val RegFiles = Vec(Reg(UInt(8 bits)),13)
    when(io.flush){
     RegFiles.map(_:=0)   
    }.elsewhen(io.en){
       RegFiles.head := io.iq
       //RegFiles.re
        for (i <- 0 until 12) {
            RegFiles(i+1) := RegFiles(i)
        }
    }
    val regRead = Reg(UInt(8 bits)) init 0 
    //regRead := io.iq
    when(io.rd){
      regRead := RegFiles(io.raddr)
    }
    io.rdata := regRead
}
showRtl(new D)